In [1]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
from tabulate import tabulate

class Dictlist(dict):
    
    def __setitem__(self, key, value):
        try:
            self[key]
        except KeyError:
            super(Dictlist, self).__setitem__(key, [])
        self[key].append(value)


class production_rule(object):
    
    result = None
    p1 = None
    p2 = None
    
    def __init__(self,result,p1,p2):
        self.result = result
        self.p1 = p1
        self.p2 = p2
    
    @property
    def get_type(self):
        return self.result
    
    @property
    def get_left(self):
        return self.p1
    
    @property
    def get_right(self):
        return self.p2

    
class Cell(object):
    productions = []
  
    def __init__(self, productions=None):
        if productions is None:
            self.productions = []
        else:
            self.productions = productions
            
    def add_production(self, result,p1,p2):
        self.productions.append(production_rule(result,p1,p2))
    
    def set_productions(self, p):
        self.productions = p
    
    @property
    def get_types(self):
        types = []
        for p in self.productions:
            types.append(p.result)
        return types
    
    @property
    def get_rules(self):       
        return self.productions


class Grammar(object):
    
    grammar_rules = Dictlist()
    parse_table = None
    length = 0
    tokens = []
    
    def __init__(self, filename):
        self.grammar_rules = Dictlist()
        self.parse_table = None
        self.length = 0
        for line in open(filename):
            a, b = line.split("->")
            self.grammar_rules[b.rstrip().strip()]=a.rstrip().strip()
        
        if len(self.grammar_rules) == 0:
            raise ValueError("No rules found in the grammar file")
        print('')
        print('Grammar file readed succesfully. Rules readed:')
        self.print_rules()
        print('')   
    
    def print_rules(self):
        for r in self.grammar_rules:
            for p in self.grammar_rules[r]:
                print(str(p) + ' --> ' + str(r))
        
    def apply_rules(self,t):
        try:
            return self.grammar_rules[t]
        except KeyError as r:
            return None
  
    def parse(self,sentence):
        self.number_of_trees = 0
        self.tokens = sentence.split()
        self.length = len(self.tokens)
        if self.length < 1:
            raise ValueError("The sentence could no be read")
        self.parse_table = [ [Cell() for x in range(self.length - y)] for y in range(self.length) ]

        for x, t in enumerate(self.tokens):
            
            r = self.apply_rules(t)
            if r == None:
                raise ValueError("The word " + str(t) + " is not in the grammar")
            else:
                for w in r: 
                    self.parse_table[0][x].add_production(w,production_rule(t,None,None),None)

        for l in range(2,self.length+1):
            for s in range(1,self.length-l+2):
                for p in range(1,l-1+1):
                    
                    t1 = self.parse_table[p-1][s-1].get_rules
                    t2 = self.parse_table[l-p-1][s+p-1].get_rules
                            
                    for a in t1:
                        for b in t2:
                            r = self.apply_rules(str(a.get_type) + " " + str(b.get_type))
                                    
                            if r is not None:
                                for w in r:
                                    print('Applied Rule: ' + str(w) + '[' + str(l) + ',' + str(s) + ']' + ' --> ' + str(a.get_type) + '[' + str(p) + ',' + str(s) + ']' + ' ' + str(b.get_type)+ '[' + str(l-p) + ',' + str(s+p) + ']')
                                    self.parse_table[l-1][s-1].add_production(w,a,b)
                               
        self.number_of_trees = len(self.parse_table[self.length-1][0].get_types)
        if  self.number_of_trees > 0:
            print("----------------------------------------")
            print('The expression IS accepted in the language')
            print("----------------------------------------")
        else:
            print("--------------------------------------------")
            print('The expression IS NOT accepted in the language')
            print("--------------------------------------------")      
        
    def get_trees(self):
        return self.parse_table[self.length-1][0].productions
          
    def print_parse_table(self):
       
        lines = [] 
        
        
        
        for row in reversed(self.parse_table):
            l = []
            for cell in row:
                l.append(cell.get_types)
            lines.append(l)
        
        lines.append(self.tokens)
        print('')
        print(tabulate(lines))
        print('')

In [4]:
g = Grammar('grammar2.txt')
g.parse('( 2 0 + 1 0 - 5 ) / ( 1 2 * 6 )')
g.print_parse_table()
trees = g.get_trees()


Grammar file readed succesfully. Rules readed:
E --> E T
E --> B P
E --> N D
N --> N D
E --> 0
N --> 0
D --> 0
E --> 1
N --> 1
D --> 1
E --> 2
N --> 2
D --> 2
E --> 3
N --> 3
D --> 3
E --> 4
N --> 4
D --> 4
E --> 5
N --> 5
D --> 5
E --> 6
N --> 6
D --> 6
E --> 7
N --> 7
D --> 7
E --> 8
N --> 8
D --> 8
E --> 9
N --> 9
D --> 9
T --> O E
P --> E C
B --> (
C --> )
O --> +
O --> -
O --> *
O --> /

Applied Rule: E[2,2] --> N[1,2] D[1,3]
Applied Rule: N[2,2] --> N[1,2] D[1,3]
Applied Rule: T[2,4] --> O[1,4] E[1,5]
Applied Rule: E[2,5] --> N[1,5] D[1,6]
Applied Rule: N[2,5] --> N[1,5] D[1,6]
Applied Rule: T[2,7] --> O[1,7] E[1,8]
Applied Rule: P[2,8] --> E[1,8] C[1,9]
Applied Rule: E[2,12] --> N[1,12] D[1,13]
Applied Rule: N[2,12] --> N[1,12] D[1,13]
Applied Rule: T[2,14] --> O[1,14] E[1,15]
Applied Rule: P[2,15] --> E[1,15] C[1,16]
Applied Rule: E[3,3] --> E[1,3] T[2,4]
Applied Rule: T[3,4] --> O[1,4] E[2,5]
Applied Rule: E[3,6] --> E[1,6] T[2,7]
Applied Rule: E[3,13] --> E[1,13] T[2,14]
App